In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle
from tqdm import tqdm
import pandas as pd
from collections import Counter
import random
import os

In [3]:
random.seed(0)
np.random.seed(0)

In [4]:
config = {
    #embedding computation
    'cleora_n_iter': 3,
    'cleora_dim': 512,
    
    #dataset preparation
    'train_test_split': 0.2,
    
    'batch_size': 256,
    'test_batch_size': 500,
    'epochs': [3],
    'alpha': [1e-4],
}

# Dataset preparation

1. Download the Facebook dataset from SNAP: https://snap.stanford.edu/data/facebook-large-page-page-network.html ->  
```bash
wget https://snap.stanford.edu/data/facebook_large.zip
```
2. Extract the dataset to ./data/facebook_large/

Other datasets from SNAP can be preprocessed similarly.

In [5]:
df = pd.read_csv("../data/facebook_large/musae_facebook_edges.csv")

In [6]:
df.head()

,id_1,id_2
0,0,18427
1,1,21708
2,1,22208
3,1,22171
4,1,6829


In [7]:
train, test = train_test_split(df, test_size=config['train_test_split'])

In [8]:
train.head()

,id_1,id_2
144023,13582,17833
137388,8090,16502
128146,11323,20700
92044,7263,16965
19,1,16260


In [9]:
train.shape, test.shape

((136801, 2), (34201, 2))

In [10]:
fb_cleora_input_clique_filename = "../fb/fb_cleora_input_clique.txt"
fb_cleora_input_star_filename = "../fb/fb_cleora_input_star.txt"
fb_lp_train_filename = "../fb/fb_lp_train.txt"
fb_lp_test_filename = "../fb/fb_lp_test.txt"
output_dir = '../output'

In [11]:
with open(fb_cleora_input_clique_filename, "w") as f_cleora_clique, open(fb_cleora_input_star_filename, "w") as f_cleora_star, open(fb_lp_train_filename, "w") as f_train:
    grouped_train = train.groupby('id_1')
    for n, (name, group) in enumerate(grouped_train):
        group_list = group['id_2'].tolist()
        group_elems = list(map(str, group_list))
        f_cleora_clique.write("{} {}\n".format(name, ' '.join(group_elems)))
        f_cleora_star.write("{}\t{}\n".format(n, name))
        for elem in group_elems:
            f_train.write("{}\t{}\n".format(name, elem))
            f_cleora_star.write("{}\t{}\n".format(n, elem))

In [12]:
with open(fb_lp_test_filename, "w") as f_test:
    grouped_test = test.groupby('id_1')
    for name, group in grouped_test:
        group_list = group['id_2'].tolist()
        group_elems = list(map(str, group_list))
        for elem in group_elems:
            f_test.write("{}\t{}\n".format(name, elem))

# Cleora training

Download an appropriate binary Cleora release from: https://github.com/Synerise/cleora/releases . 

A Linux GNU version is assumed in this example, but any other will do.

In [13]:
CLEORA_BINARY = '../src/cleora-v1.2.3-x86_64-unknown-linux-gnu'

In [14]:
import subprocess


def columns2output_filename(output_dir, columns):
    columns_split = columns.split()
    if len(columns_split) == 1 and 'reflexive' in columns:
        column_name = columns.split('::')[-1]
        return os.path.join(output_dir, f'emb__{column_name}__{column_name}.out')

    column_names = [i.split('::')[-1] for i in columns_split]
    return os.path.join(output_dir, 'emb__' + '__'.join(column_names) + '.out')


def train_cleora(dim, n_iter, columns, input_filename, output_dir):
    command = [CLEORA_BINARY,
                '--columns', columns,
                '--dimension', str(dim), 
                '-n', str(n_iter), 
                '--input', input_filename, 
                '-o', output_dir]
    subprocess.run(command, check=True)
    return columns2output_filename(output_dir, columns)

## Star expansion

In the `fb_cleora_input_star.txt` file the first column is a virtual node. The parameter `-c "transient::cluster_id node"` means that embeddings will not be created for nodes from this column. This translates to star expansion scheme.

In [15]:
# %%time
cleora_output_star_filename = train_cleora(config['cleora_dim'], config['cleora_n_iter'], "transient::cluster_id StarNode", fb_cleora_input_star_filename, output_dir)

[2024-11-17T12:27:02Z INFO  cleora] Reading args...
[src/main.rs:222] &config = Configuration {
    produce_entity_occurrence_count: true,
    embeddings_dimension: 512,
    max_number_of_iteration: 3,
    seed: None,
    prepend_field: false,
    log_every_n: 10000,
    in_memory_embedding_calculation: true,
    input: [
        "../fb/fb_cleora_input_star.txt",
    ],
    file_type: Tsv,
    output_dir: Some(
        "../output",
    ),
    output_format: TextFile,
    relation_name: "emb",
    columns: [
        Column {
            name: "cluster_id",
            transient: true,
            complex: false,
            reflexive: false,
            ignored: false,
        },
        Column {
            name: "StarNode",
            transient: false,
            complex: false,
            reflexive: false,
            ignored: false,
        },
    ],
}
[2024-11-17T12:27:02Z INFO  cleora] Starting calculation...
[src/pipeline.rs:25] &sparse_matrices = [
    SparseMatrix {
        

## Clique expansion

The `fb_cleora_input_clique.txt` file has the structure of adjacency list. The parameter `-c "complex::reflexive::node"` means that edges will be created for all cominations of nodes from each line. This translates to clique expansion scheme.

In [16]:
# %%time
cleora_output_clique_filename = train_cleora(config['cleora_dim'], config['cleora_n_iter'], "complex::reflexive::CliqueNode", fb_cleora_input_clique_filename, output_dir)

[2024-11-17T12:27:04Z INFO  cleora] Reading args...
[src/main.rs:222] &config = Configuration {
    produce_entity_occurrence_count: true,
    embeddings_dimension: 512,
    max_number_of_iteration: 3,
    seed: None,
    prepend_field: false,
    log_every_n: 10000,
    in_memory_embedding_calculation: true,
    input: [
        "../fb/fb_cleora_input_clique.txt",
    ],
    file_type: Tsv,
    output_dir: Some(
        "../output",
    ),
    output_format: TextFile,
    relation_name: "emb",
    columns: [
        Column {
            name: "CliqueNode",
            transient: false,
            complex: true,
            reflexive: true,
            ignored: false,
        },
    ],
}
[2024-11-17T12:27:04Z INFO  cleora] Starting calculation...
[src/pipeline.rs:25] &sparse_matrices = [
    SparseMatrix {
        col_a_id: 0,
        col_a_name: "CliqueNode",
        col_b_id: 1,
        col_b_name: "CliqueNode",
        edge_count: 0,
        hash_2_id: {},
        id_2_hash: [],
  

## No expansion

You can also compute Cleora without any expansion scheme by providing an input file in the edgelist format (single pair of nodes per line). Run with a simple parameter: `-c "node1 node2"`.

# Link Prediction

In the link prediction task, we train a binary classifier to distinguish real edges from fake edges. Real edges are composed of pairs of nodes from train/test set, while fake edges are built in two ways depending on whether we're training or testing:

1. In training: we draw random pairs of edges
2. In testing: we take a valid pair of nodes (nodeA-nodeB) from the testset. Then we pair nodeA to 10.000 most common nodes in the dataset. This way, we obtain 1 real and 10.000 fake examples.

We compute a Hadamard product between pairs of embeddings. As a result, we obtain a single vector for each embedding pair, which represents an approximation of node similarity. We feed the products as inputs to the classifier.

At test time, we compute ranking metrics of the correct prediction among the 10.001 given pairs of nodes. We compute the mean reciprocal rank measure (MRR) and hit rate in top 10 predictins (HR@10).

In [17]:
def read_embeddings(input_file):
    df_full = pd.read_csv(input_file, delimiter = " ", skiprows=[0], header=None, 
                     index_col=0)
    df_full = df_full.drop([1], axis=1)  
    return df_full

In [18]:
def read_train_test(embeddings):
    valid_idx = embeddings.index.to_numpy()
    train = np.loadtxt(fb_lp_train_filename, delimiter="\t", dtype=np.int32)
    test = np.loadtxt(fb_lp_test_filename, delimiter="\t", dtype=np.int32)
    
    #valid pairs of nodes
    train = train[np.isin(train[:,0], valid_idx) & np.isin(train[:,1], valid_idx)]
    test = test[np.isin(test[:,0], valid_idx) & np.isin(test[:,1], valid_idx)]
    
    #negatives for testset: top 10000 most common nodes
    all_idx = train.flatten()
    ctr = Counter(all_idx)
    negatives = ctr.most_common(10000)
    negatives = [ n[0] for n in negatives ]
    
    adjacency_dict = dict()
    for inp, out in np.vstack([test, train]):
        if inp not in adjacency_dict:
            adjacency_dict[inp] = set()
        adjacency_dict[inp].add(out)

    return train, test, negatives, adjacency_dict, valid_idx

In [19]:
batch_size = config['batch_size']
test_batch_size = config['test_batch_size']

In [20]:
for algo in [cleora_output_star_filename, cleora_output_clique_filename]:
    embeddings = read_embeddings(algo)
    train_1, test_1, negatives, adjacency_dict, valid_idx = read_train_test(embeddings)
    #for faster operation, draw only 1000 test examples
    test_ex = random.sample(list(test_1), 1000)
    
    #these are the 10.000 most common nodes selected as negatives for each valid testing node pair
    df_neg = embeddings.loc[negatives]
    neg_ids = set(df_neg.index)

    epoch = max(config['epochs'])
    for a in config['alpha']:
        #create a binary classifier outputting whether a node pair represents a valid edge (1) or not a valid edge (0)
        clf = SGDClassifier(random_state=0, loss='log_loss', alpha=a)
        for e in range(0, epoch):
            np.random.shuffle(train_1)
            
            for idx in tqdm(range(0,train_1.shape[0],batch_size)):
                #ones = real pairs of nodes
                #zeros = fake pairs of nodes
                ones=train_1[idx:min(idx+batch_size,train_1.shape[0]),:]
                
                ones_emb_in = embeddings.loc[ones[:,0]].to_numpy()
                ones_emb_out = embeddings.loc[ones[:,1]].to_numpy()
                #Hadamard
                ones = np.multiply(ones_emb_in,ones_emb_out)
                
                id_train_0_in = np.random.choice(valid_idx, size=len(ones), replace=True)
                id_train_0_out = np.random.choice(valid_idx, size=len(ones), replace=True)
    
                zeros_emb_in = embeddings.loc[id_train_0_in].to_numpy()
                zeros_emb_out = embeddings.loc[id_train_0_out].to_numpy()
                #Hadamard
                zeros = np.multiply(zeros_emb_in, zeros_emb_out)
    
                x_train = np.vstack([ones, zeros])
                y_train = [1]*len(ones) + [0]*len(ones)

                clf.partial_fit(x_train, y_train, classes=[0,1])

            if e+1 in config['epochs']:
                mrr = 0.0
                hr = 0.0
                for n, ex in enumerate(test_ex):
                    l = ex[0]
                    r = ex[1]

                    emb_l = embeddings.loc[l].to_numpy().reshape([1, -1])
                    emb_r = np.vstack((df_neg.to_numpy(), embeddings.loc[r].to_numpy()))
        
                    full_ex = np.hstack([np.repeat(emb_l, len(emb_r), axis=0), emb_r])
                    hadamard = np.multiply(emb_l, emb_r)
                    preds = clf.predict_proba(hadamard)[:,1]
                    preds = np.array(preds)

                    #do not punish for high scores of items from trainset and others from testset
                    forbidden_ex = adjacency_dict[l]
                    df_mask = [0 if (elem in forbidden_ex) else 1 for elem in neg_ids]
                    #last elem is always valid
                    df_mask.append(1)
                    preds *= df_mask
            
                    ranking = (-preds).argsort()
                    rank = np.isin(ranking, 10000).nonzero()[0][0]+1
                    mrr += 1/rank
                    hr += (rank <= 10)
                    
                    if (n+1)%100 == 0:
                        print('mrr ', mrr/(n+1), ' hr@10 ', hr/(n+1))

                print('algo: {} epochs: {} lr: {}, mrr: {}, hr@10: {}'.format(algo, str(e+1), a, mrr/len(test_ex), hr/len(test_ex)))

 59%|█████▉    | 318/535 [00:06<00:04, 49.00it/s]


KeyboardInterrupt: 